### Use Suplots with dash_bootstrap_components 
use dcc.Row and dcc.Col to form a simple grid, within a dcc.Tabs component

In [3]:
import time
import numpy as np
import pandas as pd

from flask import Flask
from flask import request
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import dash_bootstrap_components as dbc

import dash_table
import pandas as pd
import numpy as np
import json
import re
import plotly_utilities as pu
from dashapp import dashapp2 as dashapp

### Basic subplots example

In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import re

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/Mining-BTC-180.csv")

for i, row in enumerate(df["Date"]):
    p = re.compile(" 00:00:00")
    datetime = p.split(df["Date"][i])[0]
    df.iloc[i, 1] = datetime

fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}]]
)

fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["Mining-revenue-USD"],
        mode="lines",
        name="mining revenue"
    ),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["Hash-rate"],
        mode="lines",
        name="hash-rate-TH/s"
    ),
    row=2, col=1
)

fig.add_trace(
    go.Table(
        header=dict(
            values=["Date", "Number<br>Transactions", "Output<br>Volume (BTC)",
                    "Market<br>Price", "Hash<br>Rate", "Cost per<br>trans-USD",
                    "Mining<br>Revenue-USD", "Trasaction<br>fees-BTC"],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[df[k].tolist() for k in df.columns[1:]],
            align = "left")
    ),
    row=1, col=1
)
fig.update_layout(
    height=800,
    showlegend=False,
    title_text="Bitcoin mining stats for 180 days",
)

fig.show()


### simpler version of the above

In [5]:
fig2 = pu.plotly_plot(df[['Date','Number-transactions']],x_column='Date')
# fig2['data']
figm = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[
        [{"type": "table"}],
        [{"type": "scatter"}],
#         [{"type": "scatter"}]
    ]
)
tab1 =   go.Table(
        header=dict(
            values=["Date", "Number<br>Transactions", "Output<br>Volume (BTC)",
                    "Market<br>Price", "Hash<br>Rate", "Cost per<br>trans-USD",
                    "Mining<br>Revenue-USD", "Trasaction<br>fees-BTC"],
            font=dict(size=10),
            align="left"
        ),
        cells=dict(
            values=[df[k].tolist() for k in df.columns[1:]],
            align = "left")
)

figm.add_trace(tab1,row=1,col=1)
figm.add_trace(fig2['data'][0],row=2,col=1)
figm.update_layout(fig2['layout'])

# fig2['data'][0]

### Use Suplots with dash_bootstrap_components 
use dcc.Row and dcc.Col to form a simple grid, within a dcc.Tabs component

In [8]:
"""
A simple app demonstrating how to dynamically render tab content containing
dcc.Graph components to ensure graphs get sized correctly. We also show how
dcc.Store can be used to cache the results of an expensive graph generation
process so that switching tabs is fast.
"""


app = dash.Dash(external_stylesheets=[dbc.themes.LUX])

app.layout = dbc.Container(
    [
        dcc.Store(id="store"),
        html.H1("Dynamically rendered tab content"),
        html.Hr(),
        dbc.Button(
            "Regenerate graphs",
            color="primary",
            block=True,
            id="button",
            className="mb-3",
        ),
        dbc.Tabs(
            [
                dbc.Tab(label="Scatter", tab_id="scatter"),
                dbc.Tab(label="Histograms", tab_id="histogram"),
            ],
            id="tabs",
            active_tab="scatter",
        ),
        html.Div(id="tab-content", className="p-4"),
    ]
)


@app.callback(
    Output("tab-content", "children"),
    [Input("tabs", "active_tab"), Input("store", "data")],
)
def render_tab_content(active_tab, data):
    """
    This callback takes the 'active_tab' property as input, as well as the
    stored graphs, and renders the tab content depending on what the value of
    'active_tab' is.
    """
    if active_tab and data is not None:
        if active_tab == "scatter":
            return dcc.Graph(figure=data["scatter"])
        elif active_tab == "histogram":
            return dbc.Row(
                [
#                     dbc.Col(dcc.Graph(figure=data["hist_1"]), width=6),
#                     dbc.Col(dcc.Graph(figure=data["hist_2"]), width=6),
                    dbc.Col(dcc.Graph(figure=data['figm'])),
                ]
            )
        
    return "No tab selected"


@app.callback(Output("store", "data"), [Input("button", "n_clicks")])
def generate_graphs(n):
    """
    This callback generates three simple graphs from random data.
    """
    if not n:
        # generate empty graphs when app loads
        return {k: go.Figure(data=[]) for k in ["scatter", "hist_1", "hist_2"]}

    # simulate expensive graph generation process
#     time.sleep(2)

    # generate 100 multivariate normal samples
    data = np.random.multivariate_normal([0, 0], [[1, 0.5], [0.5, 1]], 100)

    scatter = go.Figure(
        data=[go.Scatter(x=data[:, 0], y=data[:, 1], mode="markers")]
    )
    figm = make_subplots(
        rows=1, cols=2,
        shared_xaxes=False,
        horizontal_spacing=0.03,
        specs=[
            [{"type": "scatter"},{"type": "scatter"}],
        ]
    )
    figm.add_traces(
        [
            go.Histogram(x=data[:, 0]),
            go.Histogram(x=data[:, 1])
        ],
        rows = [1,1],
        cols = [1,2]
    )
    
    return {"scatter": scatter, "figm":figm}


if __name__ == "__main__":
    app.run_server(debug=False, port=8884)

Dash is running on http://127.0.0.1:8884/

Dash is running on http://127.0.0.1:8884/



2021-03-17 08:50:08,321 - __main__ - INFO - Dash is running on http://127.0.0.1:8884/



2021-03-17 08:50:08,325 - __main__ - INFO -  Warning: This is a development server. Do not use app.run_server


 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.



2021-03-17 08:50:08,328 - __main__ - INFO -  in production, use a production WSGI server like gunicorn instead.



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2021-03-17 08:50:08,338 - werkzeug - INFO -  * Running on http://127.0.0.1:8884/ (Press CTRL+C to quit)
2021-03-17 08:50:10,558 - werkzeug - INFO - 127.0.0.1 - - [17/Mar/2021 08:50:10] "GET / HTTP/1.1" 200 -
2021-03-17 08:50:10,985 - werkzeug - INFO - 127.0.0.1 - - [17/Mar/2021 08:50:10] "GET /_dash-layout HTTP/1.1" 200 -
2021-03-17 08:50:10,987 - werkzeug - INFO - 127.0.0.1 - - [17/Mar/2021 08:50:10] "GET /_dash-dependencies HTTP/1.1" 200 -
2021-03-17 08:50:11,593 - werkzeug - INFO - 127.0.0.1 - - [17/Mar/2021 08:50:11] "POST /_dash-update-component HTTP/1.1" 200 -
2021-03-17 08:50:11,608 - werkzeug - INFO - 127.0.0.1 - - [17/Mar/2021 08:50:11] "POST /_dash-update-component HTTP/1.1" 200 -
2021-03-17 08:50:15,354 - werkzeug - INFO - 127.0.0.1 - - [17/Mar/2021 08:50:15] "POST /_dash-update-component HTTP/1.1" 200 -
2021-03-17 08:50:15,366 - werkzeug - INFO - 127.0.0.1 - - [17/Mar/2021 08:50:15] "POST /_dash-update-component HTTP/1.1" 200 -
2021-03-17 08:50:16,373 - werkzeug - INFO - 12

In [7]:
data = np.random.multivariate_normal([0, 0], [[1, 0.5], [0.5, 1]], 100)

figm = make_subplots(
    rows=1, cols=2,
    shared_xaxes=False,
    horizontal_spacing=0.03,
    specs=[
        [{"type": "scatter"},{"type": "scatter"}],
    ]
)
figm.add_traces(
    [
        go.Histogram(x=data[:, 0]),
        go.Histogram(x=data[:, 1])
    ],
    rows = [1,1],
    cols = [1,2]
)



In [10]:
sp_url = "https://datahub.io/core/s-and-p-500-companies/r/constituents.csv"
pd.read_csv(sp_url)

,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ABMD,Abiomed,Health Care
...,...,...,...
500,YUM,Yum! Brands Inc,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


In [12]:
import time
import tqdm
with tqdm.tqdm_notebook(np.arange(20)) as pbar:
    for i in pbar:
        pbar.set_postfix_str(f"my i = {i}")
        time.sleep(1)

/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 